In [21]:
# Standard imports
import quandl
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Plotly for better graphs
import plotly.offline as plotly
import plotly.graph_objs as go

plotly.init_notebook_mode(connected=True)

In [22]:
# Read Data from Indicators CSV
indicators = pd.read_csv('indicators.csv', sep="|")
indicators.set_index('CODE', inplace=True)

In [23]:
# Constants
# Set your Quandl API Key
quandl.ApiConfig.api_key = {YOUR API KEY}
zip_code = 94608
start_date = '2014-01-01'
end_date = '2018-07-01'
code_prefix = f'ZILLOW/Z{zip_code}_'

# Enter in the cities you want to see data for
cities = ['Berkeley, CA','Oakland, CA', 'Emeryville, CA', 'Dublin, CA', 'San Ramon, CA', 'Fremont, CA']

codes = dict(MSPAH=indicators.loc['MSPAH']['INDICATOR'],
             MLPAH=indicators.loc['MLPAH']['INDICATOR'])

In [24]:
areas_city = pd.read_csv('areas_city.csv', sep='|')
areas_city.index = areas_city['AREA']
areas_city['CODE'] = 'C' + areas_city['CODE'].astype(str)
cities_dict = areas_city.loc[cities]['CODE']

In [25]:
# Translate DataFrame into Plotly Scatter Plot
def plot_df(dataframe, title): 
    data = [go.Scatter(x=dataframe.index,
                                  y=dataframe[column],
                                  name=column) for column in dataframe.columns]
    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    return fig

In [26]:
# Get Multiple Indicators for One Location
def get_indicator_data(location, indicators):
    df_array = []
    for code, name in indicators.items():
        data = quandl.get(f'ZILLOW/{location}_{code}', start_date=start_date, end_date=end_date)
        data.rename(index=str, columns={'Value': name}, inplace=True)
        data.index = pd.to_datetime(data.index, format='%Y/%m/%d')
        df_array.append(data)
    ret = pd.concat(df_array, axis=1)
    return ret

In [27]:
# Get One Indicator for Multiple Cities
def get_city_data(cities, indicator):
    df_array = []
    for city, code in cities.items():
        quandl_db_code = f'ZILLOW/{code}_{indicator}'
        try:
            data = quandl.get(quandl_db_code, start_date=start_date, end_date=end_date)
            data.rename(index=str, columns={'Value': city}, inplace=True)
            data.index = pd.to_datetime(data.index, format='%Y/%m/%d')
            df_array.append(data)
        except:
            print('Could not fetch for:', city)
    ret = pd.concat(df_array, axis=1)
    return ret

In [28]:
dataset = get_city_data(cities_dict, 'MRP2B')

In [29]:
fig = plot_df(dataset, 'Median Rental Price 2 Bedroom')
plotly.iplot(fig)

In [30]:
# plotly.plot(fig, include_plotlyjs=False, output_type='div')

In [31]:
dataset = get_city_data(cities_dict, 'MRPAH')
fig = plot_df(dataset, 'Median Rental Price All Homes')
plotly.iplot(fig)

In [32]:
# plotly.plot(fig, include_plotlyjs=False, output_type='div')

In [33]:
dataset = get_city_data(cities_dict, 'MRPFAH')
fig = plot_df(dataset, 'Median Rental $/SqFt All Homes')
plotly.iplot(fig)

In [34]:
# plotly.plot(fig, include_plotlyjs=False, output_type='div')

In [35]:
dataset = get_city_data(cities_dict, 'MRPF2B')
fig = plot_df(dataset, 'Median Rental $/SqFt 2 Bedroom')
plotly.iplot(fig)

In [36]:
# plotly.plot(fig, include_plotlyjs=False, output_type='div')

In [37]:
dataset = get_city_data(cities_dict, 'PRRAH')
fig = plot_df(dataset, 'Price to Rent Ratio All Homes')
plotly.iplot(fig)

In [38]:
# plotly.plot(fig, include_plotlyjs=False, output_type='div')

In [39]:
dataset = get_city_data(cities_dict, 'ZRIAH')
fig = plot_df(dataset, 'Zillow Rental Index All Homes')
plotly.iplot(fig)

In [40]:
# plotly.plot(fig, include_plotlyjs=False, output_type='div')